<a href="https://colab.research.google.com/github/vishalkumar-sahu/Intelligent-Support-for-Researchers-A-Learning-Assistance-and-Plagiarism-Detection-Tool/blob/main/semantic_search_using_sbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here we are downloading and importing required libraries**

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models,util, datasets, evaluation,losses

In [ ]:
from torch.utils.data import DataLoader

**Imported two models one for training and one for comparison**

In [ ]:
model_name = 'bert-base-uncased'
# model_name = 'all-mpnet-base-v2'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
model2 = SentenceTransformer('all-mpnet-base-v2')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Importing data from test.txt for training**

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

fp=open("test.txt")
data =fp.read()
list =sent_tokenize(data)
print(list)

['Swami Vivekananda (12 January 1863 – 4 July 1902), born Narendranath Datta, was an Indian Hindu monk, philosopher, author, religious teacher, and the chief disciple of the Indian mystic Ramakrishna.', 'He was a key figure in the introduction of Vedanta and Yoga to the Western world and is credited with raising interfaith awareness, and bringing Hinduism to the status of a major world religion.', 'Born into an aristocratic Bengali Kayastha family in Calcutta, Vivekananda was inclined from a young age towards religion and spirituality.', 'He later found his guru, Ramakrishna, and became a monk.', 'After the death of Ramakrishna, Vivekananda extensively toured the Indian subcontinent, acquiring first-hand knowledge of the living conditions of Indian people in then British India.', 'Moved by their plight, he resolved to help his countrymen and found a way to travel to the United States, where he became a popular figure after the 1893 Parliament of Religions in Chicago, in which he began 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Denoising of dataset and model training**

In [ ]:
train_sentences = list

train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

train_dataloader = DataLoader(train_dataset, batch_size=8 ,shuffle=True)

train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = 1,
    weight_decay = 0,
    scheduler = 'constantlr',
    optimizer_params={'lr' : 3e-5},
    show_progress_bar = True
)

model.save('model-BERT-sentence')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.6.crossattention.self.value.bias', 'bert.encoder.layer.6.crossattention.self.query.bias', 'bert.encoder.layer.8.crossattention.output.dense.bias', 'bert.encoder.layer.11.crossattention.self.value.bias', 'bert.encoder.la

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
import numpy as np
model1 = SentenceTransformer('/content/model-BERT-sentence')


# model1 ---> is trained by us('bert-base-uncased')
# model2 ---> is pretrained model('all-mpnet-base-v2')


**Comparison of performance of trained model and pretrained model**

In [ ]:
sentence1 = ['This framework generates the sentence embeddings']
sentence2 = ['After great success at the Parliament, in the subsequent years, Vivekananda delivered hundreds of lectures across the United States']
# sentence2 = ['After great success at the Parliament, in the subsequent years, Vivekananda delivered hundreds of lectures across the United States, England and Europe, disseminating the core tenets of Hindu philosophy, and founded the Vedanta Society of New York and the Vedanta Society of San Francisco.']
# sentence1 = ['After his success in Parliament, Vivekananda gave hundreds of lectures in the US, UK and Europe in the years that followed, presented important teachings of Hindu philosophy, and founded the Vedanta Society of New York and the Vedanta Society of San Francisco.']

In [ ]:
embedding11 = model1.encode(sentence1)
embedding12 = model1.encode(sentence2)
embedding21 = model2.encode(sentence1)
embedding22 = model2.encode(sentence2)
score1 = util.pytorch_cos_sim(embedding11,embedding12)
score2 = util.pytorch_cos_sim(embedding21,embedding22)
print(score1)
print(score2)



tensor([[0.6173]])
tensor([[0.0937]])
